## Hamrobazzar Utility Price Prediction ( HamroPrice)


### Problem Statement

In the project, we are exploring different attributes of Auto (Bike) in hamrobazzar; their relations to price value and finally derive a price-prediction model. 

Since we will know the price values for our training process, it is a problem of supervised learning.


#### Target variable: Cost 

#### Features :
      1. Condition : Current bike condition (excellent, good, Like New)
      2. Anchal : 
      2. Lot No : Bike lot no. 
      3. Make Year : Bike purchased year
      4. Kilometers : Total kilometers covered by the bike

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from scipy.stats import norm

%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 8]

/home/leapfrong/workspace/envs/envpractice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leapfrong/workspace/envs/envpractice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leapfrong/workspace/envs/envpractice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('hbcsv.csv')

In [3]:
df.head()

,Brand,Name,Anchal,Type,Condition,Used For (year or month),Lot No,Price,Mileage (km / l),Engine (CC),Make Year,Kilometers
0,UnKnown,UnKnown,Bagmati,Standard,Excellent,3 years,65,135,NaN,150,2015,NaN
1,yamaha,UnKnown,Bagmati,Sports,Good/Fair,5 months,32,85000,NaN,153,2010,27000
2,UnKnown,UnKnown,Bagmati,Sports,Excellent,3 years,67,155,NaN,153,2015,23000
3,UnKnown,UnKnown,Bagmati,Sports,Excellent,NaN,67,115000,NaN,125,2015,NaN
4,yamaha,UnKnown,Bagmati,Standard,Excellent,NaN,32,68000,NaN,150,2009,NaN


In [4]:
df.shape

(3471, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3471 entries, 0 to 3470
Data columns (total 12 columns):
Brand                       3471 non-null object
Name                        3471 non-null object
Anchal                      3471 non-null object
Type                        3471 non-null object
Condition                   3281 non-null object
Used For (year or month)    2139 non-null object
Lot No                      3470 non-null object
Price                       3471 non-null int64
Mileage  (km / l)           0 non-null float64
Engine (CC)                 3086 non-null object
Make Year                   2502 non-null object
Kilometers                  2669 non-null object
dtypes: float64(1), int64(1), object(10)
memory usage: 325.5+ KB


In [6]:
df.index

RangeIndex(start=0, stop=3471, step=1)

In [7]:
# rename cols
columns = ['brand', 'bike_name', 'area', 'type', 'condition', 'months_used', 'lot_number',
 'cost', 'mileage', 'engine_cc', 'make_year', 'kilometers']
df.columns = columns



In [8]:
df.head()

,brand,bike_name,area,type,condition,months_used,lot_number,cost,mileage,engine_cc,make_year,kilometers
0,UnKnown,UnKnown,Bagmati,Standard,Excellent,3 years,65,135,NaN,150,2015,NaN
1,yamaha,UnKnown,Bagmati,Sports,Good/Fair,5 months,32,85000,NaN,153,2010,27000
2,UnKnown,UnKnown,Bagmati,Sports,Excellent,3 years,67,155,NaN,153,2015,23000
3,UnKnown,UnKnown,Bagmati,Sports,Excellent,NaN,67,115000,NaN,125,2015,NaN
4,yamaha,UnKnown,Bagmati,Standard,Excellent,NaN,32,68000,NaN,150,2009,NaN


In [9]:
selected_columns = ['brand', 'condition', 'months_used','lot_number',
                   'engine_cc', 'make_year', 'kilometers', 'cost']

df = df[selected_columns]
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,UnKnown,Excellent,3 years,65,150,2015,NaN,135
1,yamaha,Good/Fair,5 months,32,153,2010,27000,85000
2,UnKnown,Excellent,3 years,67,153,2015,23000,155
3,UnKnown,Excellent,NaN,67,125,2015,NaN,115000
4,yamaha,Excellent,NaN,32,150,2009,NaN,68000


In [10]:
# convert months used from years to total months (default value to 12 months)
import math
import re

word_to_num = {'one':1,
          'two':2,
          'three':3,
          'four': 4,
          'five': 5}

def get_int_value_from_string(x):
    """
    for ex: 1year..
    """
    try:
        int_val = int(re.search(r'\d+', x).group())
        str_val = x[1:]
        if str_val.lower().startswith('y'):
            return int_val * 12
        else:
            return int_val
    except:
        return 12
    
def get_total_months(x):
    if x:
        try:
            math.isnan(x)
            return 12
        except:
            pass
            
        x = str(x.strip().lower())
        
        if x.startswith('few') or x.startswith('some'):
            return 36
        if x.startswith('a year') and x.endswith('more'):
            return 24
        
        if '+' in x:
            int_val = int(re.search(r'\d+', x).group())
            return (int_val + 1) * 12
        check_list = x.split(' ')
        
        if '/' in check_list[0] or '-' in check_list[0]:
            sep = '/' if '/' in check_list[0] else '-'
            if len(check_list) == 1:
                slash_values = list(map(int, re.findall(r'\d+', check_list[0])))
                print('slahs valeus', slash_values)
                try:
                    return int(slash_values[1]) * 12
                except:
                    return int(slash_values[0]) * 12
                    
            slash_values = check_list[0].split(sep)
            if check_list[1].lower().startswith('y'):
                if slash_values[1].isdigit():
                    return int(slash_values[1])* 12
                else:
                    return slash_values[0]* 12
            else:
                if slash_values[1].isdigit():
                    return int(slash_values[1])
                else:
                    return slash_values[0]
                
            
        if check_list[0].lower() in word_to_num:
            return word_to_num[check_list[0].lower()]
        
        if len(check_list) == 1:
            return get_int_value_from_string(x)
        
        elif check_list[1].lower() == 'above':
            return get_int_value_from_string(check_list[0].lower())
            
        else:    
            if check_list[1].lower().startswith('y'):
                no_of_years = check_list[0]
                if no_of_years.isdigit():
                    no_of_years = int(no_of_years)
                    return no_of_years * 12
                else:
                    orig = float(no_of_years)
                    whole = math.floor(orig)    # whole = 5.0
                    frac = orig - whole  
                    frac_months = int(frac * 12)
                    return int(whole) * 12 + frac_months

            else:
                if check_list[1].lower().startswith('m'):
                    if check_list[0].isdigit():
                        return int(check_list[0])
                    else:
                        int_vals = list(map(int, re.findall(r'\d+', check_list[0])))
                        if len(int_vals) > 1:
                            total_months = int_vals[0] * 12 + int_vals[1]
                            return total_months
                else:
                    return 12
#                 else:
#                     return int(check_list[0])
    else:
        return 12
            
    

In [11]:
df['months_used'] = df['months_used'].apply(lambda x: get_total_months(x))

slahs valeus [3, 4]
slahs valeus [3, 4]
slahs valeus [2074, 10, 1, 2075]
slahs valeus [2069, 5, 3]
slahs valeus [2]
slahs valeus [4, 5]
slahs valeus [4, 5]


In [12]:
df.tail()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
3466,vespa,Like New (used few times),12,11,150,NaN,NaN,110000
3467,UnKnown,Excellent,36,62,230,2014,36000,130000
3468,UnKnown,Excellent,12,55,1200,2012,6000,2800000
3469,honda,Good/Fair,12,7,100,NaN,NaN,50000
3470,mahindra,Good/Fair,36,49,125,2012,7000,65000


In [13]:
df.months_used.isnull().sum()

5

In [14]:
df.dropna(
    subset=['months_used'],
    how='any', inplace = True)

### Handling Missing Values

In [15]:
# drop rows if all the column values are null
df.dropna(how='all').shape

(3466, 8)

### Fill Missing Values for brand

In [16]:
# let us see which of the brand is occurring frequently
df['brand'].value_counts()

UnKnown          1727
bajaj             569
honda             423
yamaha            207
suzuki            102
tvs               100
royal enfield      89
hero               73
crossfire          51
benelli            39
mahindra           37
vespa              19
um                 18
hartford            8
reiju               2
apollo              1
aprilla             1
Name: brand, dtype: int64

In [17]:
df.brand.isnull().sum()

0

In [18]:
# first fill 'UnKnown' brand with nan
df['brand'] = df['brand'].replace('UnKnown', np.nan)
# df['brand'] = df['brand'].replace('NaN', np.nan)

In [19]:
df['brand'].value_counts()

bajaj            569
honda            423
yamaha           207
suzuki           102
tvs              100
royal enfield     89
hero              73
crossfire         51
benelli           39
mahindra          37
vespa             19
um                18
hartford           8
reiju              2
aprilla            1
apollo             1
Name: brand, dtype: int64

In [20]:
df.brand.isnull().sum()

1727

In [21]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3461, 3462, 3463, 3464, 3465, 3466, 3467, 3468, 3469, 3470],
           dtype='int64', length=3466)

In [22]:
# we can see bajaj is the most ocurring brand
df['brand'].fillna(value='bajaj', inplace=True)

In [23]:
df.brand.isnull().sum()

0

In [24]:
df['brand'].value_counts()

bajaj            2296
honda             423
yamaha            207
suzuki            102
tvs               100
royal enfield      89
hero               73
crossfire          51
benelli            39
mahindra           37
vespa              19
um                 18
hartford            8
reiju               2
aprilla             1
apollo              1
Name: brand, dtype: int64

### Fill Missing Values for condition

In [25]:
# values count in condition
df['condition'].value_counts()

Excellent                    1955
Like New (used few times)     955
Good/Fair                     366
Not Working                     1
Name: condition, dtype: int64

In [26]:
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,NaN,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,NaN,115000
4,yamaha,Excellent,12,32,150,2009,NaN,68000


In [27]:
# df['condition'] = df['condition'].replace('NaN', np.nan)
df['condition'].fillna(value=np.nan, inplace=True)

In [28]:
# total 190 null entries in condition column
total_null_count_2 = df.condition.isnull().sum()
total_null_count_2

189

In [29]:
condition_random_choices = ['Excellent','Like New (used few times)', 'Good/Fair']

In [30]:
df['condition'] = df['condition'].fillna(
    pd.Series(np.random.choice(condition_random_choices, size=len(df.index))))

### Fill Missing Values for engine/cc

In [31]:
# remove empty kilometers rows
df[df.engine_cc == '']

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost


In [32]:
df['engine_cc'] = df['engine_cc'].replace('', np.nan)

In [33]:
df.engine_cc.isnull().sum()

385

In [34]:
# we can use brand name to calculate the CC value.
# we collect all possible engine_cc values for given brand and return mode
import math
def remove_non_numeric_char(cc):
    if len(cc) == 0:
        return np.nan
    if not cc.isdigit():
        try:
            cc = float(cc)
            cc = math.ceil(cc)
        except ValueError:
            print ("Not a float")
    result = re.sub('[^0-9]','',str(cc))
    return result

def get_engine_cc_based_on_brand(brand):
    engine_cc_series = df[df['brand']==brand]['engine_cc']
    return engine_cc_series.mode()[0]
    

In [35]:
get_engine_cc_based_on_brand('bajaj')

'150'

In [36]:
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,NaN,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,NaN,115000
4,yamaha,Excellent,12,32,150,2009,NaN,68000


In [37]:
df['engine_cc'] =  df.apply(
    lambda x: get_engine_cc_based_on_brand(x['brand'])
    if pd.isnull(x['engine_cc']) else x['engine_cc'], axis=1)

In [38]:
df.engine_cc.isnull().sum()

0

In [39]:
df['engine_cc'] = df['engine_cc'].apply(lambda x : remove_non_numeric_char(x))

Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float


### Fill Missing Values for make_year

In [40]:
df.shape

(3466, 8)

In [41]:
df.lot_number.isnull().sum()

1

In [42]:
df.dropna(
    subset=['lot_number'],
    how='any', inplace = True)

In [43]:
df.shape

(3465, 8)

In [44]:
df['lot_number'] = df['lot_number'].str.replace('.', '')

In [45]:
 df = df[df['lot_number'].map(len) < 4]

In [46]:
df[df.lot_number.map(len) == 4]

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost


In [47]:
df[df.lot_number == '125']

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
3361,bajaj,Like New (used few times),12,125,125,NaN,NaN,90000


In [48]:
df['lot_number'] = df['lot_number'].astype(str)
# df['lot_number'] = df[df['lot_number'] != 'Ga']
# df['lot_number'] = df['lot_number'].astype(int)
# .astype(int)
# df['lot_number'] = df['lot_number'].apply(lambda x: x.isnumeric())

# df.lot_number = pd.to_numeric(df.lot_number, errors='coerce')
# df.info()

def check_numeric(value):
    try:
        return int(value)
    except:
        return np.nan

df['lot_number'] = df['lot_number'].apply(lambda x: check_numeric(x))
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460 entries, 0 to 3470
Data columns (total 8 columns):
brand          3460 non-null object
condition      3460 non-null object
months_used    3460 non-null object
lot_number     3452 non-null float64
engine_cc      3460 non-null object
make_year      2494 non-null object
kilometers     2664 non-null object
cost           3460 non-null int64
dtypes: float64(1), int64(1), object(6)
memory usage: 243.3+ KB


,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65.0,150,2015,NaN,135
1,yamaha,Good/Fair,5,32.0,153,2010,27000,85000
2,bajaj,Excellent,36,67.0,153,2015,23000,155
3,bajaj,Excellent,12,67.0,125,2015,NaN,115000
4,yamaha,Excellent,12,32.0,150,2009,NaN,68000


In [49]:
df.lot_number.isnull().sum()

8

In [50]:
df.dropna(
    subset=['lot_number'],
    how='any', inplace = True)

In [51]:
df.lot_number.isnull().sum()

0

In [52]:
df['lot_number'] = df['lot_number'].astype(int)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3452 entries, 0 to 3470
Data columns (total 8 columns):
brand          3452 non-null object
condition      3452 non-null object
months_used    3452 non-null object
lot_number     3452 non-null int64
engine_cc      3452 non-null object
make_year      2488 non-null object
kilometers     2659 non-null object
cost           3452 non-null int64
dtypes: int64(2), object(6)
memory usage: 242.7+ KB


In [54]:
df.dropna(
    subset=['lot_number'],
    how='any', inplace = True)

In [55]:
def fill_na_non_numeric_char(make_year):
    try:
        if make_year.isdigit():
            return make_year
        else:
            try:
                make_year = float(make_year)
                make_year = math.ceil(make_year)
            except ValueError:
                print ("str", make_year)
                return np.nan
    except:
        return np.nan
            
    result = re.sub('[^0-9]','',str(make_year))
    return result

In [56]:
df['make_year'] = df['make_year'].apply(lambda x : fill_na_non_numeric_char(x))

str 3 year
str A.20
str -
str -
str 2 ye
str n
str 3 year


In [57]:
# we can estimate the make_year based on lot_number and 
# looking up corresponing make_year for a given lot_number

def get_make_year_based_on_lotnumber(lot_number):
    make_year_series = df[df['lot_number']==lot_number]['make_year']
    
    if make_year_series.empty:
        make_year = df.ix[
            (df['lot_number']-lot_number).abs().argsort()[0]]['make_year']
        
        return make_year
    
    if make_year_series.dropna().empty:
        return get_make_year_based_on_lotnumber(int(lot_number) - 1)
    
    return make_year_series.mode()[0]

    

In [58]:
df['make_year'] =  df.apply(
    lambda x: get_make_year_based_on_lotnumber(x['lot_number'])
    if pd.isnull(x['make_year']) else x['make_year'], axis=1)

/home/leapfrong/workspace/envs/envpractice/lib/python3.5/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [59]:
df.make_year.isnull().sum()

0

In [60]:
df.dropna(
    subset=['make_year'],
    how='any', inplace = True)

In [61]:
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,NaN,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,NaN,115000
4,yamaha,Excellent,12,32,150,2009,NaN,68000


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3452 entries, 0 to 3470
Data columns (total 8 columns):
brand          3452 non-null object
condition      3452 non-null object
months_used    3452 non-null object
lot_number     3452 non-null int64
engine_cc      3452 non-null object
make_year      3452 non-null object
kilometers     2659 non-null object
cost           3452 non-null int64
dtypes: int64(2), object(6)
memory usage: 242.7+ KB


In [63]:
df = df[df.make_year != '015.016']

In [64]:
df['make_year'] = df['make_year'].astype(str).astype(int)

### Fill Missing Values for kilometers

In [65]:
df.kilometers.isnull().sum()

793

In [66]:
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,NaN,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,NaN,115000
4,yamaha,Excellent,12,32,150,2009,NaN,68000


In [67]:
# let us fill up null kilometers based on months_used (mode value)

def get_kilometers_based_on_months_used(months):
    km_series = df[df['months_used']==months]['kilometers']
    if km_series.empty:
        return get_kilometers_based_on_months_used(months - 1)
    
    if km_series.dropna().empty:
        return get_kilometers_based_on_months_used(months - 1)
    return km_series.mode()[0]


In [68]:
df['kilometers'] = df.apply(
    lambda x : get_kilometers_based_on_months_used(x['months_used'])
    if pd.isnull(x['kilometers']) else x['kilometers'], axis=1)

In [69]:
df.kilometers.isnull().sum()

0

In [70]:
df[df['kilometers'].isnull()]

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost


In [71]:
df.head(10)

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,30000,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,12000,115000
4,yamaha,Excellent,12,32,150,2009,12000,68000
5,bajaj,Like New (used few times),12,65,220,2015,16800,164000
6,bajaj,Excellent,36,71,109,2015,13.5k,130000
7,bajaj,Good/Fair,48,9,109,2014,19000,150000
8,bajaj,Like New (used few times),2,96,200,2018,3000,305000
9,honda,Excellent,12,81,160,2017,13000,185000


In [72]:
# let us remove rows having kilometers < 1000, (removing outliers)
df = df[df['kilometers'].map(len) > 4]

In [73]:
df.tail()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
3464,yamaha,Excellent,12,4,100,2009,29000,45000
3465,honda,Excellent,96,23,150,200,24000,80000
3466,vespa,Like New (used few times),12,11,150,2016,12000,110000
3467,bajaj,Excellent,36,62,230,2014,36000,130000
3469,honda,Good/Fair,12,7,100,2012,12000,50000


In [74]:
df['kilometers'] = df['kilometers'].str.replace('+', '')
df['kilometers'] = df['kilometers'].str.replace(',', '')

In [75]:
# replace kilimeter abbr. (k) with multiplication of 1000

def remove_kilo_abbr(kilo):
    if kilo:
        kilo = re.sub(r"(\d+)(k)", r"\g<1>000", kilo.replace('.', ''))
        return kilo.strip()

def remove_thousand_abbr(kilo):
    print('kilo is', kilo)
    kilo_values = kilo.split(" ")
    if len(kilo_values)==2:
        if kilo_values[1].lower().strip().startswith('th'):
            return str(int(kilo_values[0]) * 1000)
    else:
        int_val = int(re.search(r'\d+', kilo).group())
        return str(int_val * 1000)

def remove_non_numeric_char(kilo):
    result = re.sub('[^0-9]','', kilo)
    return result

In [76]:
df['kilometers'] = df['kilometers'].apply(lambda x: remove_kilo_abbr(x))
df['kilometers'] = df['kilometers'].apply(lambda x: remove_non_numeric_char(x))

In [77]:
df.kilometers.isnull().sum()

0

In [78]:
df[df.kilometers == '40 tho']

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost


In [79]:
df['kilometers'] = df['kilometers'].apply(
    lambda x: remove_thousand_abbr(x) if 'th' in x else x)


In [80]:
df.head()

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
0,bajaj,Excellent,36,65,150,2015,30000,135
1,yamaha,Good/Fair,5,32,153,2010,27000,85000
2,bajaj,Excellent,36,67,153,2015,23000,155
3,bajaj,Excellent,12,67,125,2015,12000,115000
4,yamaha,Excellent,12,32,150,2009,12000,68000


In [81]:
df.shape

(2483, 8)

In [82]:
# remove empty kilometers rows
df = df[df.kilometers.map(len) !=0]

In [83]:
df[df.engine_cc =='']

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost
834,bajaj,Excellent,10,88,,2017,10000,245000
2372,bajaj,Good/Fair,120,10,,2000,25000,40000
2975,bajaj,Excellent,8,80,,2016,10000,420000


In [84]:
df = df[df.engine_cc !='']

In [85]:
df['condition'] = df['condition'].astype(str)
df['kilometers'] = df['kilometers'].astype(str).astype(int)
df['months_used'] = df['months_used'].astype(str).astype(int)
df['engine_cc'] = df['engine_cc'].astype(str).astype(int)

In [86]:
# add new feature travel_years(number of years)
import datetime

this_year = datetime.datetime.today().year

df['auto_age'] = df['make_year'].apply(lambda x: this_year - x)

In [87]:
df.head(10)

,brand,condition,months_used,lot_number,engine_cc,make_year,kilometers,cost,auto_age
0,bajaj,Excellent,36,65,150,2015,30000,135,3
1,yamaha,Good/Fair,5,32,153,2010,27000,85000,8
2,bajaj,Excellent,36,67,153,2015,23000,155,3
3,bajaj,Excellent,12,67,125,2015,12000,115000,3
4,yamaha,Excellent,12,32,150,2009,12000,68000,9
5,bajaj,Like New (used few times),12,65,220,2015,16800,164000,3
6,bajaj,Excellent,36,71,109,2015,135000,130000,3
7,bajaj,Good/Fair,48,9,109,2014,19000,150000,4
9,honda,Excellent,12,81,160,2017,13000,185000,1
11,bajaj,Excellent,11,88,220,2017,14000,245000,1


In [88]:
# condition is a categorical variable
df['condition'] = df['condition'].astype('category')
# brand is a categorical variable
df['brand'] = df['brand'].astype('category')

In [89]:
df['condition'].cat.categories

Index(['Excellent', 'Good/Fair', 'Like New (used few times)', 'Not Working'], dtype='object')

In [90]:
df['brand'].cat.categories

Index(['apollo', 'bajaj', 'benelli', 'crossfire', 'hartford', 'hero', 'honda',
       'mahindra', 'reiju', 'royal enfield', 'suzuki', 'tvs', 'um', 'vespa',
       'yamaha'],
      dtype='object')

In [91]:
df['cost'].describe()

count    2.478000e+03
mean     1.453154e+06
std      4.757636e+07
min      1.800000e+01
25%      8.500000e+04
50%      1.400000e+05
75%      2.150000e+05
max      2.147484e+09
Name: cost, dtype: float64

In [92]:
df = df[df.cost >= 10000]

In [93]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

# Use only one feature : CollegeGPA
# labour_data_gpa = labour_data.loc[:, ['collegeGPA']] or labour_data[['collegeGPA']]
auto_months = df[['months_used']]
# salary as a dependent variable
# labour_data_salary = labour_data.loc[:, 'Salary']
auto_cost = df[['cost']]
# Split the data into training/testing sets
months_train, months_test, cost_train, cost_test = train_test_split(
    auto_months, auto_cost)

# Create linear regression object
regression = LinearRegression(fit_intercept = True)

# Train the model using the training sets (first parameter is x )
regression.fit(months_train, cost_train)
# salary_X_train.size, salary_Y_train.size


/home/leapfrong/workspace/envs/envpractice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leapfrong/workspace/envs/envpractice/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leapfrong/workspace/envs/envpractice/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Coefficients

In [94]:
regression.coef_, regression.intercept_

(array([[3.10514195e-07]]), array([185498.43084404]))

### predict the cost

In [95]:
cost_predict = regression.predict(months_test)
cost_predict[:10]

array([[185498.43084776],
       [185498.43085149],
       [185498.43084776],
       [185498.43084776],
       [185498.43086267],
       [185498.43084776],
       [185498.43084435],
       [185498.43084776],
       [185498.43084963],
       [185498.43084776]])

Compare the predicted value with the actual value of cost

In [96]:
cost_test[:10]

,cost
1825,70000
2116,350000
1826,70000
3345,200000
321,150000
3234,185000
877,140000
753,90000
3374,190000
1537,160000


In [97]:
regression.score(
    months_test, cost_test), regression.score(months_train, cost_train)

(-0.00294263823986074, 1.8204600614191158e-05)

## Multiple Linear Regression - (Linear Regression with three independent variable)

In [98]:
feature_cols = ['months_used', 'kilometers', 'auto_age']
X = df[feature_cols]
y = df.cost
# Split the data into training/testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y)

lm = LinearRegression()
lm.fit(x_train, y_train)

# print intercept and coefficients
print (lm.intercept_)
print (lm.coef_)

750973.7898184969
[-4.76013291e-06 -7.77906163e-02 -3.73996908e+02]


In [99]:
# feature coeffs.
list(zip(feature_cols, lm.coef_))

[('months_used', -4.760132909539037e-06),
 ('kilometers', -0.07779061630279471),
 ('auto_age', -373.99690757467533)]

In [100]:
cost_predict_mul = lm.predict(x_test)
cost_predict_mul[:10]

array([749370.0991669 , 737256.10545178, 747727.51320714, 746692.29600387,
       741459.3484161 , 746895.7555049 , 738796.50194111, 746066.96138397,
       748747.77423647, 748281.03053866])

In [101]:
y_test[:10]

1263    210000
1403     75000
946     350000
826      90000
1861     55000
241     180000
3353     40000
2077     80000
639     135000
450     167000
Name: cost, dtype: int64

In [102]:
lm.score(x_test, y_test)

-0.0011798294914411311

### Handling 'Categorical input variables'  : brand and condition

In [636]:
# create X and y
feature_cols = ['months_used', 'auto_age', 'kilometers', 'brand', 'condition']

X = df[feature_cols]
X_months = df[['months_used']]
X_age = df[['auto_age']]
X_kilo = df[['kilometers']]
X_brand = pd.get_dummies(df['brand'])                     
X_condition = pd.get_dummies(df['condition'])
X_new = pd.concat([X_months,X_age, X_kilo, X_brand, X_condition], axis = 1)

X_new.head()

,months_used,auto_age,kilometers,apollo,bajaj,benelli,crossfire,hartford,hero,honda,...,royal enfield,suzuki,tvs,um,vespa,yamaha,Excellent,Good/Fair,Like New (used few times),Not Working
1,5,8,27000,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,12,3,12000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,12,9,12000,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
5,12,3,16800,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,36,3,135000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [637]:

y_new = df.cost
# Split the data into training/testing sets
x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(X_new, y_new)

lm_cat = LinearRegression()
lm_cat.fit(x_cat_train, y_cat_train)

# print intercept and coefficients
print (lm_cat.intercept_)
print (lm_cat.coef_)

-6415.294031998608
[-3.25511690e-05 -1.04508244e+03 -2.46977427e-01  1.33597562e+06
  2.26348252e+06 -4.78600625e+05  3.00656133e+05 -4.28981688e+04
 -5.60134869e+05 -5.29760202e+05 -6.34457304e+05  1.37940799e+06
 -4.45840896e+05 -6.05171355e+05 -3.35010660e+05 -1.00523630e+06
 -2.36494700e+05 -4.05917175e+05  1.58601697e+06 -1.15823593e+06
 -1.05133183e+06  6.23550789e+05]


In [638]:
predict_cat = lm_cat.predict(x_cat_test)

In [639]:
predict_cat[:10]

array([3835940.13941902, 1201726.58284965, 3837840.4995969 ,
       1004485.78798858, 1255881.063773  , 3831684.34964282,
       3831741.52295735, 1240088.25283028, 3826934.6065742 ,
       1166778.27912036])

In [640]:
y_cat_train.head()

3244     50000
2451     75000
2169    245000
1655    230000
1767    150000
Name: cost, dtype: int64

In [641]:
lm_cat.score(x_cat_test, y_cat_test)

-312.3610826943585